In [4]:
import sys
sys.path.append('/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat')
from app.imports.runtime import *
from app.scheduler.mozzarella import *
from app.scheduler.mozzarella.boiling_plan.boiling_plan import (
    cast_boiling_plan,
)


In [36]:
fn = DebugConfig.abs_path(
    "app/data/static/samples/inputs/mozzarella/2021-02-28 План по варкам.xlsx"
)
boiling_plan_df = cast_boiling_plan(fn)
start_times = {LineName.WATER: "02:00", LineName.SALT: "06:00"}

boilings = make_boilings(boiling_plan_df, first_group_id=1)
schedule = make_schedule_from_boilings(
boilings, start_times=start_times
)

In [37]:
boilings = schedule['master']['boiling']
boilings = list(sorted(boilings, key=lambda b: b.x[0]))

In [38]:
boilings[0].props.all()

In [53]:
values = [[b1['pouring']['first']['termizator'].x[0],
           b1['pouring']['first']['termizator'].y[0],
          b1.props['boiling_group_df'].iloc[0]['group_id'],
          b1.props['boiling_group_df'].iloc[0]['line'].name]
          for b1, b2 in iter_pairs(boilings, method='any_suffix')]

In [54]:
df = pd.DataFrame(values, columns=['x', 'y', 'group_id', 'line_name'])

In [55]:
df

In [66]:
df['time_till_next_boiling'] = (df['x'].shift(-1) - df['y']).fillna(0).astype(int)

In [67]:
df

In [152]:
df['is_water_done'] = df['line_name']
df['is_water_done'] = np.where(df['is_water_done'] == 'Моцарелла в воде', True, np.nan)
df['is_water_done'] = df['is_water_done'].fillna(method='bfill')
df['is_water_done'] = df['is_water_done'].shift(-1).fillna(0)
df['is_water_done'] = df['is_water_done'].astype(bool)
df['is_water_done'] = ~df['is_water_done']
df

In [76]:
cast_t('12:00')

In [170]:
def is_cleaning_combination_fit(cleaning_combination):
    separators = [-1] + list(cleaning_combination) + [20]
    print(separators)
    for s1, s2 in iter_pairs(separators):
        group = df.loc[s1 + 1:s2]
        
        group_length = group.iloc[-1]['y'] - group.iloc[0]['x']
        if group_length > cast_t('12:00'):
            return False
    return True

In [173]:
is_cleaning_combination_fit([])

In [174]:
n_cleanings = 0

In [175]:
list(itertools.combinations(range(3), 2))

In [176]:
list(itertools.combinations(range(3), 0))

In [177]:
available_combinations = [combo for combo in itertools.combinations(range(len(df) - 1), n_cleanings) if is_cleaning_combination_fit(combo)]

In [178]:
len(available_combinations)

In [154]:
values1 = [[combo, sum(df.loc[s]['time_till_next_boiling'] for s in combo), df.loc[combo[0]]['is_water_done']] for combo in available_combinations]
df1 = pd.DataFrame(values1, columns=['combo', 'total_time_till_next_boilings', 'is_water_done'])
df1

In [157]:
df1 = df1.sort_values(by=['total_time_till_next_boilings'], ascending=False)
df1 = df1.sort_values(by=['is_water_done'], ascending=False)
df1

In [159]:
combo = df1.iloc[0]
combo